In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns


In [ ]:
pd.__version__


In [44]:
'''
Shell command for downloading data
'''
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv
'''找出2010年美國各州及地區人口密度排名  -Python資料科學學習手冊p.159'''

'找出2010年美國各州及地區人口密度排名'

In [2]:
# 讀取csv檔，建立pd的dataframe
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-areas.csv')

In [276]:
pop

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
...,...,...,...,...
2539,USA,total,2010,309326295.0
2540,USA,under18,2011,73902222.0
2541,USA,total,2011,311582564.0
2542,USA,under18,2012,73708179.0


In [43]:
# 先檢視基本資料狀態
print(pop.head())
print('-'*50)
print(areas.head())
print('-'*50
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
--------------------------------------------------
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
--------------------------------------------------
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [100]:
# 多對1合併，先讓pop這個dataframe取得州的全名

# 直接合併預設會採用inner join 缺值資料會被刪除，先採用outer join取得所有資料
merged0 = pd.merge(pop,abbrevs,how='outer',left_on='state/region',right_on='abbreviation')
merged0.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [101]:
# 去掉'abbreviation'這個欄位
#merged1 = merged0.drop(['abbreviation'],'columns')   #  'columns' >　整欄去掉。　可以用axis=１,1表示
merged1 = merged0.drop(columns=['abbreviation'] ) 
merged1.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [98]:
# 檢查第一個合併dataframe的缺值及處裡
merged1.isnull().any() # 發現population、state 2個欄位有缺值

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [103]:
#　population有缺值合理，先檢視state缺值狀況
state_null = merged1[merged1['state'].isnull()]  # abbreviation 有96筆缺值資料

# 利用groupby ,以欄名 'state/region'為key分群檢視，發現PR、USA兩個地區沒有對應的州名
state_null.groupby('state/region').count()  

,ages,year,population,state
state/region,,,,
PR,48,48,28,0
USA,48,48,48,0


In [160]:
# 處裡state欄位缺值，幫PR、USA兩個state/region補上缺值

# 將'state/region'欄位中名稱為PR的欄位，指定state值為'Puerto Rico' 
merged1.loc[merged1['state/region']=='PR','state'] = 'Puerto Rico' 

# 將'state/region'欄位中名稱為USA的欄位，指定state值為'United States' 
merged1.loc[merged1['state/region']=='USA','state'] = 'United States'  

#再檢視一次資料缺值狀況，確認state欄位缺值是否都被處理了
merged1.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [261]:
# 檢視population欄位缺值狀況

#利用groupby ,以欄名 'state/region'為key分群檢視，發現缺值的欄位都在PR
pop_null = merged1[merged1['population'].isnull()]
pop_null.groupby('state/region').count()   

# 調出'state/region' == PR 資料，發現populattion缺值的資料跟年代有關
pop_null.groupby('state/region').count() 

#檢視PR地區 population值為null的資訊，發現1999年以前族群量缺乏資料 
#merged1.loc[merged1['state/region']=='PR',['year','population']]  #僅選擇year,population兩個欄位
pr = merged1.loc[merged1['state/region']=='PR']
pr_null = pr.loc[pr['population'].isnull()] 
pr_null['year'].describe() 


count      20.000000
mean     1994.500000
std         2.946898
min      1990.000000
25%      1992.000000
50%      1994.500000
75%      1997.000000
max      1999.000000
Name: year, dtype: float64

In [379]:
# 利用state欄位與areas dataframe合併
# 這邊使用left或outer效果是一樣的，避免直接用inner取交集，使缺值資料消失
merged2 = pd.merge(merged1,areas, how='left')   

# 檢查新產生的dataframe是否有其他缺值，發現合併後area欄位有缺值
# 由於area有缺值，所以接下來檢視area欄位出現空值時，state欄位有哪些值
merged2.isnull().any()

# 檢視area欄位缺值的狀況，發現都是region為USA的資料(可用count檢視資料筆數)
areaNull = merged2['area (sq. mi)'].isnull()
stateSeries = merged2['state'][areaNull]  #轉變為一為陣列
stateSeries.unique()   
 
 

array(['United States'], dtype=object)

In [390]:
# 簡單的拋棄缺值
final = merged2.dropna() #去除包括population、area欄位有缺值的資料 
final = final.set_index('state')  # 將state設為index 方便後面創立density欄位時索引
final
#final = merged2.dropna(subset=['area (sq. mi)'])  #指定只去除特定欄位的缺值

#final.groupby('state')['area (sq. mi)'].describe() #檢查資料分布狀況 

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,under18,2012,1117489.0,52423.0
Alabama,AL,total,2012,4817528.0,52423.0
Alabama,AL,under18,2010,1130966.0,52423.0
Alabama,AL,total,2010,4785570.0,52423.0
Alabama,AL,under18,2011,1125763.0,52423.0
...,...,...,...,...,...
Puerto Rico,PR,under18,2010,896945.0,3515.0
Puerto Rico,PR,under18,2011,869327.0,3515.0
Puerto Rico,PR,total,2011,3686580.0,3515.0


In [406]:
# 將density加入final的dataframe內
final['density'] = final['population']/final['area (sq. mi)']

# 篩選2010年且所有人口的資料
data2010 = final[ (final['year']==2010) & (final['ages']=='total') ]

# dataframe的排序法
data2010.sort_values(by=['density'], ascending=False).head()

# vu0v 先取出density的Series再做排序
# data2010['density'].sort_values(ascending=False).head()

,state/region,ages,year,population,area (sq. mi),density
state,,,,,,
District of Columbia,DC,total,2010,605125.0,68.0,8898.897059
Puerto Rico,PR,total,2010,3721208.0,3515.0,1058.665149
New Jersey,NJ,total,2010,8802707.0,8722.0,1009.253268
Rhode Island,RI,total,2010,1052669.0,1545.0,681.339159
Connecticut,CT,total,2010,3579210.0,5544.0,645.600649


In [371]:
final['population']



0       1117489.0
1       4817528.0
2       1130966.0
3       4785570.0
4       1125763.0
          ...    
2491     896945.0
2492     869327.0
2493    3686580.0
2494     841740.0
2495    3651545.0
Name: population, Length: 2476, dtype: float64